# OpenAI API - Entry Level Demo

Welcome to the **Entry Level** OpenAI API tutorial! This notebook will guide you through:

1. **Setting up your environment** - Installing dependencies and configuring API keys
2. **Making your first API call** - Basic chat completions
3. **Understanding the response** - Parsing and using API responses
4. **Basic parameters** - Temperature, max_tokens, and model selection
5. **Simple conversation** - Multi-turn chat basics

## Prerequisites
- Python 3.8+
- An OpenAI API key (get one at https://platform.openai.com/api-keys)

---

## Reference Documentation
- [OpenAI Quickstart](https://platform.openai.com/docs/quickstart)
- [Chat Completions API](https://platform.openai.com/docs/api-reference/chat)

## 1. Environment Setup

First, let's install the required packages and set up our API key.

In [1]:
# Install the OpenAI Python SDK (run once)
# !pip install openai python-dotenv

# Import required libraries
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file (if you have one)
load_dotenv()

# =============================================================================
# GLOBAL CONFIGURATION
# =============================================================================
# Set the model to use throughout this notebook
MODEL = "gpt-4o-mini"  # Change to "gpt-4o" for more capable model
# =============================================================================

# Initialize the OpenAI client
# Option 1: Set your API key as an environment variable OPENAI_API_KEY
# Option 2: Pass it directly (not recommended for production)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # or replace with your key: "sk-..."
)

print("✓ OpenAI client initialized successfully!")
print(f"✓ Using model: {MODEL}")

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

## 2. Your First API Call - Basic Chat Completion

The Chat Completions API is the core of OpenAI's offerings. It takes a list of messages and returns a model-generated response.

**Key concepts:**
- `model`: Which AI model to use (e.g., "gpt-4o", "gpt-4o-mini")
- `messages`: A list of conversation messages with roles (system, user, assistant)
- `role`: Who is "speaking" - system sets behavior, user is you, assistant is the AI

In [10]:
# Make your first API call!
response = client.chat.completions.create(
    model=MODEL,  # Using the global MODEL variable
    messages=[
        {
            "role": "user",
            "content": "Hello! What can you help me with today?"
        }
    ]
)

# Extract and print the response
assistant_message = response.choices[0].message.content
print("Assistant:", assistant_message)

NotFoundError: Error code: 404 - {'error': {'message': 'Your organization must be verified to use the model `gpt-5-mini`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

## 3. Understanding the Response Object

The API returns a structured response with metadata about the completion. Let's explore it!

In [ ]:
# Let's examine the full response structure
print("=== Full Response Object ===")
print(f"ID: {response.id}")
print(f"Model: {response.model}")
print(f"Created: {response.created}")

print("\n=== Usage Statistics ===")
print(f"Prompt tokens: {response.usage.prompt_tokens}")
print(f"Completion tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")

print("\n=== Choice Details ===")
choice = response.choices[0]
print(f"Finish reason: {choice.finish_reason}")
print(f"Message role: {choice.message.role}")
print(f"Message content: {choice.message.content[:100]}...")

=== Full Response Object ===
ID: chatcmpl-D5znl9W6e0bkU1SU7prLqhR0WbEnp
Model: gpt-4o-mini-2024-07-18
Created: 1770321577

=== Usage Statistics ===
Prompt tokens: 17
Completion tokens: 56
Total tokens: 73

=== Choice Details ===
Finish reason: stop
Message role: assistant
Message content: Hello! I'm here to help you with a variety of topics. Whether you have questions about general knowl...


## 4. Using System Messages

The **system message** sets the behavior and personality of the assistant. It's like giving the AI instructions before the conversation starts.

In [ ]:
# Using a system message to customize assistant behavior
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful coding tutor who explains concepts simply and uses analogies. Keep responses concise."
        },
        {
            "role": "user",
            "content": "What is a variable in programming?"
        }
    ]
)

print("Assistant (as coding tutor):")
print(response.choices[0].message.content)

Assistant (as coding tutor):
Think of a variable in programming like a labeled box where you can store information. Just like you might label a box "Toys" or "Books," in programming, you give a variable a name, like `age` or `score`. 

You can put something inside that box (like the number 10 for `age`), and you can change what's inside it whenever you need to. The variable acts as a placeholder, allowing you to refer to the information without needing to remember the exact value.


## 5. Key Parameters: Temperature and Max Tokens

**Temperature** (0-2): Controls randomness in responses
- Lower (0-0.3): More focused, deterministic, factual
- Higher (0.7-1.0): More creative, varied, imaginative

**Max Tokens**: Limits the length of the response

In [ ]:
# Comparing different temperature settings
prompt = "Write a one-sentence story about a robot."

# Low temperature - more predictable/focused
low_temp_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2,
    max_tokens=100
)

# High temperature - more creative/varied
high_temp_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=1.0,
    max_tokens=100
)

print("Low Temperature (0.2) - More focused:")
print(low_temp_response.choices[0].message.content)
print("\nHigh Temperature (1.0) - More creative:")
print(high_temp_response.choices[0].message.content)

Low Temperature (0.2) - More focused:
In a world where emotions were forbidden, a lonely robot discovered an old book of poetry and, for the first time, felt the warmth of longing in its metallic heart.

High Temperature (1.0) - More creative:
In a world where emotions were obsolete, a curious little robot discovered an ancient book of poetry and, for the first time, yearned to feel.


## 6. Multi-Turn Conversations

To have a conversation with context, you include previous messages in each request. The model doesn't remember previous calls - you must provide the conversation history!

In [ ]:
# Building a simple conversation with memory
conversation_history = [
    {"role": "system", "content": "You are a friendly assistant who remembers our conversation."}
]

def chat(user_message):
    """Send a message and get a response, maintaining conversation history."""
    # Add user message to history
    conversation_history.append({"role": "user", "content": user_message})
    
    # Make API call with full history
    response = client.chat.completions.create(
        model=MODEL,
        messages=conversation_history
    )
    
    # Extract assistant message
    assistant_message = response.choices[0].message.content
    
    # Add assistant response to history
    conversation_history.append({"role": "assistant", "content": assistant_message})
    
    return assistant_message

# Have a multi-turn conversation
print("User: My name is Alex and I love pizza.")
print("Assistant:", chat("My name is Alex and I love pizza."))

print("\nUser: What's my name?")
print("Assistant:", chat("What's my name?"))

print("\nUser: What food did I mention?")
print("Assistant:", chat("What food did I mention?"))

User: My name is Alex and I love pizza.
Assistant: Nice to meet you, Alex! Pizza is a delicious choice. Do you have a favorite type or topping?

User: What's my name?
Assistant: Your name is Alex!

User: What food did I mention?
Assistant: You mentioned that you love pizza!


## 7. Basic Error Handling

Always handle potential API errors gracefully. Common issues include rate limits, invalid API keys, and network problems.

In [ ]:
from openai import APIError, RateLimitError, APIConnectionError

def safe_chat_completion(messages, model=None):
    """Make an API call with proper error handling."""
    if model is None:
        model = MODEL  # Use global MODEL if not specified
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response.choices[0].message.content
    
    except RateLimitError as e:
        print(f"Rate limit exceeded. Please wait and try again: {e}")
        return None
    
    except APIConnectionError as e:
        print(f"Connection error. Check your internet: {e}")
        return None
    
    except APIError as e:
        print(f"API error occurred: {e}")
        return None

# Test the safe function
result = safe_chat_completion([
    {"role": "user", "content": "Say hello!"}
])
if result:
    print("Success:", result)

Success: Hello! How can I assist you today?


## 8. Available Models

OpenAI offers several models with different capabilities and price points. Here are the main ones:

In [ ]:
# Common models and their use cases
models_info = """
╔════════════════════════════════════════════════════════════════════╗
║                     OPENAI MODELS OVERVIEW                         ║
╠════════════════════════════════════════════════════════════════════╣
║ GPT-4o          │ Latest flagship model, multimodal (text+vision)  ║
║                 │ Best for: Complex tasks, vision, high accuracy    ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ GPT-4o-mini     │ Fast and affordable, great for most tasks        ║
║                 │ Best for: Quick responses, cost-effective usage  ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ o1 / o1-mini    │ Reasoning models for complex problem solving     ║
║                 │ Best for: Math, coding, logical reasoning        ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ GPT-4-turbo     │ Previous generation flagship                     ║
║                 │ Best for: Legacy applications, specific needs    ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ GPT-3.5-turbo   │ Older, very fast and cheap                       ║
║                 │ Best for: Simple tasks, high volume, low cost    ║
╚════════════════════════════════════════════════════════════════════╝
"""
print(models_info)

# List available models from your account
print("\nYour available models (first 10):")
models = client.models.list()
for i, model in enumerate(models.data[:10]):
    print(f"  - {model.id}")


╔════════════════════════════════════════════════════════════════════╗
║                     OPENAI MODELS OVERVIEW                         ║
╠════════════════════════════════════════════════════════════════════╣
║ GPT-4o          │ Latest flagship model, multimodal (text+vision)  ║
║                 │ Best for: Complex tasks, vision, high accuracy    ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ GPT-4o-mini     │ Fast and affordable, great for most tasks        ║
║                 │ Best for: Quick responses, cost-effective usage  ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ o1 / o1-mini    │ Reasoning models for complex problem solving     ║
║                 │ Best for: Math, coding, logical reasoning        ║
╠─────────────────┼──────────────────────────────────────────────────╣
║ GPT-4-turbo     │ Previous generation flagship                     ║
║                 │ Best for: Legacy applications, specific needs    ║
╠───

---

## Summary

Congratulations! You've learned the fundamentals of the OpenAI API:

1. **Client Setup**: Initialize the OpenAI client with your API key
2. **Chat Completions**: The core API for generating text responses
3. **Message Roles**: system (instructions), user (your input), assistant (AI output)
4. **Response Parsing**: Extract content and metadata from responses
5. **Parameters**: Control output with temperature and max_tokens
6. **Multi-turn Conversations**: Maintain context by including message history
7. **Error Handling**: Gracefully handle API errors

### Next Steps
Move on to **02_middle_level_openai_api.ipynb** to learn:
- Streaming responses
- Function calling / Tools
- Vision capabilities
- Structured outputs